In [1]:
import pandas as pd
import numpy as np
import os
from pandasql import sqldf
from scipy import stats
import time
import networkx as nx

In [ ]:
# Load original and matched datasets

dataset_loc = '...' # Insert path to dataset
matched_dataset_loc = '...' # Insert path to matched dataset

bnf_df = pd.read_csv(dataset_loc)
bnf_df['eventdate'] = pd.to_datetime(bnf_df['eventdate'])
match = pd.read_csv(matched_dataset_loc)

In [9]:
# Create dictionary from dateindex to matchindex
indexes = dict(zip(bnf_df['dateindex'].to_list(), bnf_df['matchindex'].to_list()))

In [2]:
# 2-prescription model

bnflist = [] # Insert list of of lists where each inner list consists of 2 prescriptions to use as 1 node (first 2 in list)
# and a prescription to use as target node (last element in list) 
result = []

for bnf1, bnf2, bnf3 in bnflist:
    
    bnf_pair_result = []
    bnf_pair_result.append(bnf1)
    bnf_pair_result.append(bnf2)
    bnf_pair_result.append(bnf3)

    # Get dataset for patients with D1/D2 within a year of each other
    q = """ 
    SELECT MIN(dateindex) dateindex, patid, MIN(eventdate) eventdate FROM (
    SELECT a.dateindex, a.matchindex, a.patid, a.bnfcode, a.eventdate, a.num, a.gender, a.yob, a.gen_ethnicity, a.bnf, a.bnfchapter, a.bnfsection, a.bnfparagraph, a.gen_ethnicity_int,
    b.eventdate eventdate2, b.bnf bnf2, b.bnfchapter bnfchapter2, b.bnfsection bnfsection2
    FROM bnf_df AS a
    JOIN bnf_df as b
    ON a.patid = b.patid
    AND ABS(julianday(b.eventdate) - julianday(a.eventdate))<365
    AND a.bnfsection = {}
    AND b.bnfsection = {}) AS T
    GROUP BY patid
    """.format(bnf1, bnf2)
    df = sqldf(q)

    # Get corresponding matched patients to the selected patients
    match_df = match[match['matchindex'].isin([indexes.get(key) for key in df['dateindex']])]

    # Count how many of the D1/D2 patients go on to be prescribed with D3 within 5 years
    q = """
    SELECT *, ep2_icd IS NOT NULL FROM (
    SELECT a.*, b.patid AS patid_dup, b.bnfsection as ep2_icd, b.eventdate as ep2_start
    FROM df AS a
    LEFT JOIN bnf_df AS b
    ON a.patid = b.patid
    AND b.eventdate > a.eventdate
    AND julianday(b.eventdate) - julianday(a.eventdate) < 1825
    AND b.bnfsection = {}
    LIMIT 100000000) AS T
    LIMIT 100000000;
    """.format(bnf3)
    df2 = sqldf(q)
    df2['eventdate'] = pd.to_datetime(df2['eventdate'])
    
    # Just select 1 row per patient
    df3 = df2.loc[df2.groupby('patid').eventdate.idxmin()]
    # Count total number of D1/D2 patients who go on to be prescribed with D3 within 5 years
    num_D1_to_D2 = df3['ep2_icd IS NOT NULL'].sum()
    # Count total number of D1/D2 patients
    Total_D1_patients = len(df3)

    bnf_pair_result.append(Total_D1_patients)
    bnf_pair_result.append(num_D1_to_D2)

    # Count how many of the matched patients go on to be prescribed with D3 within 5 years 
    q = """
    SELECT *, ep2_icd IS NOT NULL FROM (
    SELECT a.*, b.patid AS patid_dup, b.bnfsection as ep2_icd, b.eventdate as ep2_start
    FROM match_df AS a
    LEFT JOIN bnf_df AS b
    ON a.patid = b.patid
    AND b.eventdate > a.eventdate
    AND julianday(b.eventdate) - julianday(a.eventdate) < 1825
    AND b.bnfsection = {}
    LIMIT 100000000) AS T
    LIMIT 100000000;
    """.format(bnf3)
    df2 = sqldf(q)
    df2['eventdate'] = pd.to_datetime(df2['eventdate'])
    
    # Just select 1 row per patient
    df3_match = df2.loc[df2.groupby('patid').eventdate.idxmin()]
    # Filter out rows where no match was found
    df3_match = df3_match[df3_match["patid"]!=-1]
    # Count total number of matched patients who go on to be prescribed with D3 within 5 years
    num_D1_to_D2_match = df3_match['ep2_icd IS NOT NULL'].sum()
    # Count total number of matched patients
    Total_matched_patients = len(df3_match)

    # Calculate relative risk
    RR = (num_D1_to_D2*Total_matched_patients)/(num_D1_to_D2_match*D1len)

    bnf_pair_result.append(Total_matched_patients)
    bnf_pair_result.append(num_D1_to_D2_match)
    bnf_pair_result.append(RR)
    print(bnf_pair_result)

    result.append(bnf_pair_result)
    
# Save csv
# pd.DataFrame(result).to_csv('BNF2nodeResults.csv', index=False)

In [ ]:
# 3-prescription model

bnflist = [] # Insert list of of lists where each inner list consists of 3 prescriptions to use as 1 node (first 3 in list)
# and a prescription to use as target node (last element in list) 
result = []

for bnf1, bnf2, bnf3, bnf4 in bnflist:
    
    bnf_pair_result = []
    bnf_pair_result.append(bnf1)
    bnf_pair_result.append(bnf2)
    bnf_pair_result.append(bnf3)
    
    # Get dataset for patients with D1/D2/D3 within a year of each other
    q = """ 
    SELECT MIN(dateindex) dateindex, patid, MIN(eventdate) eventdate FROM (
    SELECT a.dateindex, a.matchindex, a.patid, a.bnfcode, a.eventdate, a.num, a.gender, a.yob, a.gen_ethnicity, a.bnf, a.bnfchapter, a.bnfsection, a.bnfparagraph, a.gen_ethnicity_int,
    b.eventdate eventdate2, b.bnf bnf2, b.bnfchapter bnfchapter2, b.bnfsection bnfsection2
    c.eventdate eventdate3, c.bnf bnf3, c.bnfchapter bnfchapter3, c.bnfsection bnfsection3
    FROM bnf_df AS a
    JOIN bnf_df as b
    JOIN bnf_df as c
    ON a.patid = b.patid
    AND a.patid = c.patid
    AND ABS(julianday(b.eventdate) - julianday(a.eventdate))<365
    AND ABS(julianday(c.eventdate) - julianday(a.eventdate))<365
    AND a.bnfsection = {}
    AND b.bnfsection = {}
    AND c.bnfsection = {}) AS T
    GROUP BY patid
    """.format(bnf1, bnf2, bnf3)
    df = sqldf(q)

    # Get corresponding matched patients to the selected patients
    match_df = match[match['matchindex'].isin([indexes.get(key) for key in df['dateindex']])]

    # Count how many of the D1/D2/D3 patients go on to be prescribed with D4 within 5 years
    q = """
    SELECT *, ep2_icd IS NOT NULL FROM (
    SELECT a.*, b.patid AS patid_dup, b.bnfsection as ep2_icd, b.eventdate as ep2_start
    FROM df AS a
    LEFT JOIN bnf_df AS b
    ON a.patid = b.patid
    AND b.eventdate > a.eventdate
    AND julianday(b.eventdate) - julianday(a.eventdate) < 1825
    AND b.bnfsection = {}
    LIMIT 100000000) AS T
    LIMIT 100000000;
    """.format(bnf4)
    df2 = sqldf(q)
    df2['eventdate'] = pd.to_datetime(df2['eventdate'])
    
    # Just select 1 row per patient
    df3 = df2.loc[df2.groupby('patid').eventdate.idxmin()]
    # Count total number of D1/D2/D3 patients who go on to be prescribed with D4 within 5 years
    num_D1_to_D2 = df3['ep2_icd IS NOT NULL'].sum()
    # Count total number of D1/D2/D3 patients
    Total_D1_patients = len(df3)

    bnf_pair_result.append(Total_D1_patients)
    bnf_pair_result.append(num_D1_to_D2)
    
    # Count how many of the matched patients go on to be prescribed with D4 within 5 years 
    q = """
    SELECT *, ep2_icd IS NOT NULL FROM (
    SELECT a.*, b.patid AS patid_dup, b.bnfsection as ep2_icd, b.eventdate as ep2_start
    FROM match_df AS a
    LEFT JOIN bnf_df AS b
    ON a.patid = b.patid
    AND b.eventdate > a.eventdate
    AND julianday(b.eventdate) - julianday(a.eventdate) < 1825
    AND b.bnfsection = {}
    LIMIT 100000000) AS T
    LIMIT 100000000;
    """.format(bnf4)
    df2 = sqldf(q)
    df2['eventdate'] = pd.to_datetime(df2['eventdate'])
      
    # Just select 1 row per patient
    df3_match = df2.loc[df2.groupby('patid').eventdate.idxmin()]
    # Filter out rows where no match was found
    df3_match = df3_match[df3_match["patid"]!=-1]
    # Count total number of matched patients who go on to be diagnosed with D4 within 5 years
    num_D1_to_D2_match = df3_match['ep2_icd IS NOT NULL'].sum()
    # Count total number of matched patients
    Total_matched_patients = len(df3_match)

    # Calculate relative risk
    RR = (num_D1_to_D2*Total_matched_patients)/(num_D1_to_D2_match*D1len)

    bnf_pair_result.append(Total_matched_patients)
    bnf_pair_result.append(num_D1_to_D2_match)
    bnf_pair_result.append(RR)
    print(bnf_pair_result)

    result.append(bnf_pair_result)
    
# Save csv    
# pd.DataFrame(result).to_csv('BNF3nodeResults.csv', index=False)

In [ ]:
# 4-prescription model

bnflist = [] # Insert list of of lists where each inner list consists of 4 diagnoses to use as 1 node (first 4 in list)
# and a diagnosis to use as target node (last element in list) 
result = []

for bnf1, bnf2, bnf3, bnf4, bnf5 in bnflist:
    
    bnf_pair_result = []
    bnf_pair_result.append(bnf1)
    bnf_pair_result.append(bnf2)
    bnf_pair_result.append(bnf3)

    # Get dataset for patients with D1/D2/D3/D4 within a year of each other
    q = """ 
    SELECT MIN(dateindex) dateindex, patid, MIN(eventdate) eventdate FROM (
    SELECT a.dateindex, a.matchindex, a.patid, a.bnfcode, a.eventdate, a.num, a.gender, a.yob, a.gen_ethnicity, a.bnf, a.bnfchapter, a.bnfsection, a.bnfparagraph, a.gen_ethnicity_int,
    b.eventdate eventdate2, b.bnf bnf2, b.bnfchapter bnfchapter2, b.bnfsection bnfsection2
    c.eventdate eventdate3, c.bnf bnf3, c.bnfchapter bnfchapter3, c.bnfsection bnfsection3
    d.eventdate eventdate4, d.bnf bnf4, d.bnfchapter bnfchapter4, d.bnfsection bnfsection4
    FROM bnf_df AS a
    JOIN bnf_df as b
    JOIN bnf_df as c
    JOIN bnf_df as d
    ON a.patid = b.patid
    AND a.patid = c.patid
    AND a.patid = d.patid
    AND ABS(julianday(b.eventdate) - julianday(a.eventdate))<365
    AND ABS(julianday(c.eventdate) - julianday(a.eventdate))<365
    AND ABS(julianday(d.eventdate) - julianday(a.eventdate))<365
    AND a.bnfsection = {}
    AND b.bnfsection = {}
    AND c.bnfsection = {}
    AND d.bnfsection = {}) AS T
    GROUP BY patid
    """.format(bnf1, bnf2, bnf3, bnf4)
    df = sqldf(q)

    # Get corresponding matched patients to the selected patients
    match_df = match[match['matchindex'].isin([indexes.get(key) for key in df['dateindex']])]

    # Count how many of the D1/D2/D3/D4 patients go on to be prescribed with D5 within 5 years
    q = """
    SELECT *, ep2_icd IS NOT NULL FROM (
    SELECT a.*, b.patid AS patid_dup, b.bnfsection as ep2_icd, b.eventdate as ep2_start
    FROM df AS a
    LEFT JOIN bnf_df AS b
    ON a.patid = b.patid
    AND b.eventdate > a.eventdate
    AND julianday(b.eventdate) - julianday(a.eventdate) < 1825
    AND b.bnfsection = {}
    LIMIT 100000000) AS T
    LIMIT 100000000;
    """.format(bnf5)
    df2 = sqldf(q)
    df2['eventdate'] = pd.to_datetime(df2['eventdate'])
    
    # Just select 1 row per patient
    df3 = df2.loc[df2.groupby('patid').eventdate.idxmin()]
    # Count total number of D1/D2/D3/D4 patients who go on to be prescribed with D5 within 5 years
    num_D1_to_D2 = df3['ep2_icd IS NOT NULL'].sum()
    # Count total number of D1/D2/D3/D4 patients
    Total_D1_patients = len(df3)

    bnf_pair_result.append(Total_D1_patients)
    bnf_pair_result.append(num_D1_to_D2)

    # Count how many of the matched patients go on to be diagnosed with D5 within 5 years 
    q = """
    SELECT *, ep2_icd IS NOT NULL FROM (
    SELECT a.*, b.patid AS patid_dup, b.bnfsection as ep2_icd, b.eventdate as ep2_start
    FROM match_df AS a
    LEFT JOIN bnf_df AS b
    ON a.patid = b.patid
    AND b.eventdate > a.eventdate
    AND julianday(b.eventdate) - julianday(a.eventdate) < 1825
    AND b.bnfsection = {}
    LIMIT 100000000) AS T
    LIMIT 100000000;
    """.format(bnf5)
    df2 = sqldf(q)
    df2['eventdate'] = pd.to_datetime(df2['eventdate'])
     
    # Just select 1 row per patient
    df3_match = df2.loc[df2.groupby('patid').eventdate.idxmin()]
    # Filter out rows where no match was found
    df3_match = df3_match[df3_match["patid"]!=-1]
    # Count total number of matched patients who go on to be diagnosed with D5 within 5 years
    num_D1_to_D2_match = df3_match['ep2_icd IS NOT NULL'].sum()
    # Count total number of matched patients
    Total_matched_patients = len(df3_match)

    # Calculate relative risk
    RR = (num_D1_to_D2*Total_matched_patients)/(num_D1_to_D2_match*D1len)

    bnf_pair_result.append(Total_matched_patients)
    bnf_pair_result.append(num_D1_to_D2_match)
    bnf_pair_result.append(RR)
    print(bnf_pair_result)

    result.append(bnf_pair_result)
    
# Save csv
# pd.DataFrame(result).to_csv('BNF4nodeResults.csv', index=False)